In [1]:
!pip install torch torchvision

    100% |████████████████████████████████| 519.5MB 114kB/s eta 0:00:01   16% |█████▎                          | 85.6MB 43.1MB/s eta 0:00:11    64% |████████████████████▌           | 333.2MB 56.0MB/s eta 0:00:04
    100% |████████████████████████████████| 61kB 30.0MB/s ta 0:00:01
You are using pip version 10.0.1, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [3]:
!mkdir train
!mkdir tests

mkdir: cannot create directory ‘train’: File exists


In [4]:
import numpy as np
import sklearn
import pickle
from scipy import sparse as sp
import pandas as pd
import random

In [5]:
pref_data = sp.load_npz("pref.npz")
#conf_data = sp.load_npz("confi.npz")

In [40]:
data_coo = pref_data.tocoo(copy=False)

In [52]:
a = []

In [41]:
df = pd.DataFrame({'userID': data_coo.row, 'itemID': data_coo.col, 'rating': data_coo.data})[['userID', 'itemID', 'rating']] \
.reset_index(drop=True)

In [57]:
for _, row in df.iterrows():
    a.append([row[0],row[1],row[2]])
    break

a

[userID    0
 itemID    0
 rating    1
 Name: 0, dtype: int64, userID    0
 itemID    0
 rating    1
 Name: 0, dtype: int64, userID    0
 itemID    0
 rating    1
 Name: 0, dtype: int64, [0, 0, 1], [0, 0, 1]]

In [63]:
list_data = [[0,-1] for i in range(146976)]
for index, row in df.iterrows():
    list_data[row[0]][0]+=1
    list_data[row[0]][1] = index
    if(index%10000 ==0):
        print(index)
    
    

0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000
320000
330000
340000
350000
360000
370000
380000
390000
400000
410000
420000
430000
440000
450000
460000
470000
480000
490000
500000
510000
520000
530000
540000
550000
560000
570000
580000
590000
600000
610000
620000
630000
640000
650000
660000
670000
680000
690000
700000
710000
720000
730000
740000
750000
760000
770000
780000
790000
800000
810000
820000
830000
840000
850000
860000
870000
880000
890000
900000
910000
920000
930000
940000
950000
960000
970000
980000
990000
1000000
1010000
1020000
1030000
1040000
1050000
1060000
1070000
1080000
1090000
1100000
1110000
1120000
1130000
1140000
1150000
1160000
1170000
1180000
1190000
1200000
1210000
1220000
1230000
1240000
1250000
1260000
1270000
1280000
1290000
1300000
1310000
1320000
1330000
1340000
1350000
1360000
1370000
1380000
13

10350000
10360000
10370000
10380000
10390000
10400000
10410000
10420000
10430000
10440000
10450000
10460000
10470000
10480000
10490000
10500000
10510000
10520000
10530000
10540000
10550000
10560000
10570000
10580000
10590000
10600000
10610000
10620000
10630000
10640000
10650000
10660000
10670000
10680000
10690000
10700000
10710000
10720000
10730000
10740000
10750000
10760000
10770000
10780000
10790000
10800000
10810000
10820000
10830000
10840000
10850000
10860000
10870000
10880000
10890000
10900000
10910000
10920000
10930000
10940000
10950000
10960000
10970000
10980000
10990000
11000000
11010000
11020000
11030000
11040000
11050000
11060000
11070000
11080000
11090000
11100000
11110000
11120000
11130000
11140000
11150000
11160000
11170000
11180000
11190000
11200000
11210000
11220000
11230000
11240000


In [64]:
for i in range(146976):
    if list_data[i][0] == 0:
        print(i)
        break

15108


In [65]:
len()

userID    146975
itemID      4225
rating         0
Name: 11248214, dtype: int64

In [68]:
cnt = len(df)
add_data = []
for x in list_data:
    if x[0]==1:
        add_data.append(df.iloc[x[1]])
        cnt+=1

df=df.append(add_data)

In [70]:
df.iloc[-1]

userID    118759
itemID      3195
rating         1
Name: 9396852, dtype: int64

In [71]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(df, test_size=0.2,stratify=df[['userID']])

In [74]:
train.iloc[-2]

userID    50199
itemID     1349
rating        1
Name: 6731757, dtype: int64

In [75]:
train.to_csv("train.txt", sep='\t', index = False, header=False)

In [76]:
test.to_csv("test.txt", sep='\t', index = False, header=False)

In [77]:
!mv train.txt ./train
!mv test.txt ./tests

In [78]:
!python run.py --gpu_ids 0 \
--path_to_train_data train \
--path_to_eval_data tests \
--hidden_layers 512,512,1024 \
--non_linearity_type selu \
--batch_size 128 \
--logdir model_save \
--drop_prob 0.8 \
--optimizer momentum \
--lr 0.005 \
--weight_decay 0 \
--aug_step 1 \
--noise_prob 0 \
--num_epochs 12 \
--summary_frequency 1000

Namespace(aug_step=1, batch_size=128, constrained=False, drop_prob=0.8, gpu_ids='0', hidden_layers='512,512,1024', logdir='model_save', lr=0.005, noise_prob=0.0, non_linearity_type='selu', num_epochs=12, optimizer='momentum', path_to_eval_data='tests', path_to_train_data='train', save_every=3, skip_last_layer_nl=False, summary_frequency=1000, weight_decay=0.0)
GPU is available.
Loading training data
146644
Data loaded
Total items found: 146644
Vector dim: 4431
Loading eval data
/home/ubuntu/DeepRecommender-master/reco_encoder/model/model.py:60: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  weight_init.xavier_uniform(w)
/home/ubuntu/DeepRecommender-master/reco_encoder/model/model.py:72: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  weight_init.xavier_uniform(w)
******************************
******************************
[4431, 512, 512, 1024]
Dropout drop probability: 0.8
Encoder pass:
torch.

In [39]:
import gc
gc.collect()

0

In [80]:
ind = 0
for index,row in df.iterrows():
    if row[0] == 18951:
        ind = index
        break

In [130]:
ind = 3891170
ind

3891170

In [131]:
tdf = pd.DataFrame(columns=['userID','itemID','rating'])
tdf.head()

,userID,itemID,rating


In [132]:
while True:
    if(df.iloc[ind][0] == 18951):
        tdf = tdf.append(df.iloc[ind])
        ind+=1
    else:
        break

In [133]:
tdf = tdf.astype('int64')

In [148]:
len(tdf)

376

In [134]:
tdf.to_csv("test_18951.txt", sep='\t', index = False, header=False)

In [153]:
l = []
for i in range(4434):
    l.append([18951,i,0])

In [154]:
testdf = pd.DataFrame(l)

In [156]:
tdf.to_csv("testdf_18951.txt", sep='\t', index = False, header=False)

In [135]:
!python infer.py \
--path_to_train_data train \
--path_to_eval_data tests \
--hidden_layers 512,512,1024 \
--non_linearity_type selu \
--save_path model_save/model.epoch_11 \
--drop_prob 0.8 \
--predictions_path preds.txt

Namespace(constrained=False, drop_prob=0.8, hidden_layers='512,512,1024', non_linearity_type='selu', path_to_eval_data='tests', path_to_train_data='train', predictions_path='preds.txt', save_path='model_save/model.epoch_11', skip_last_layer_nl=False)
GPU is available.
Loading training data
146644
Data loaded
Total items found: 146644
Vector dim: 4431
Loading eval data
/home/ubuntu/DeepRecommender-master/reco_encoder/model/model.py:60: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  weight_init.xavier_uniform(w)
/home/ubuntu/DeepRecommender-master/reco_encoder/model/model.py:72: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  weight_init.xavier_uniform(w)
******************************
******************************
[4431, 512, 512, 1024]
Dropout drop probability: 0.8
Encoder pass:
torch.Size([512, 4431])
torch.Size([512])
torch.Size([512, 512])
torch.Size([512])
torch.Size([1024, 512])
torch.Size(

In [141]:
pred = pd.read_csv('preds.txt',delimiter='\t',header=None)

In [142]:
pred.head()

,0,1,2,3
0,18951,3582,1.026844,1.0
1,18951,3906,1.018442,1.0
2,18951,4025,0.995049,1.0
3,18951,3164,1.012115,1.0
4,18951,3328,0.987053,1.0


In [150]:
pred.sort_values(1,inplace=True,ascending=False)
tdf.sort_values('itemID',inplace=True,ascending=False)

In [158]:
print(pred)

         0     1         2    3
257  18951  4251  1.019388  1.0
16   18951  4224  1.009129  1.0
5    18951  4223  0.993901  1.0
292  18951  4222  0.978424  1.0
263  18951  4217  0.964515  1.0
185  18951  4182  0.992785  1.0
138  18951  4179  1.015135  1.0
2    18951  4025  0.995049  1.0
141  18951  4024  1.034578  1.0
43   18951  3960  1.015859  1.0
28   18951  3959  1.011680  1.0
351  18951  3944  1.007758  1.0
298  18951  3943  0.978963  1.0
112  18951  3942  1.040879  1.0
260  18951  3940  1.025038  1.0
172  18951  3939  0.989226  1.0
88   18951  3938  0.978047  1.0
106  18951  3935  1.026593  1.0
42   18951  3934  1.000157  1.0
114  18951  3933  1.007156  1.0
80   18951  3932  1.001165  1.0
192  18951  3931  1.007964  1.0
226  18951  3930  0.974374  1.0
56   18951  3915  0.969437  1.0
14   18951  3914  0.963195  1.0
39   18951  3913  1.002358  1.0
1    18951  3906  1.018442  1.0
205  18951  3900  0.992872  1.0
25   18951  3899  0.991758  1.0
143  18951  3898  0.961280  1.0
..     .

In [157]:
print(tdf)

         userID  itemID  rating
3891545   18951    4251       1
3891544   18951    4224       1
3891543   18951    4223       1
3891542   18951    4222       1
3891541   18951    4217       1
3891540   18951    4182       1
3891539   18951    4179       1
3891538   18951    4111       0
3891537   18951    4025       1
3891536   18951    4024       1
3891535   18951    3960       1
3891534   18951    3959       1
3891533   18951    3944       1
3891532   18951    3943       1
3891531   18951    3942       1
3891530   18951    3940       1
3891529   18951    3939       1
3891528   18951    3938       1
3891527   18951    3935       1
3891526   18951    3934       1
3891525   18951    3933       1
3891524   18951    3932       1
3891523   18951    3931       1
3891522   18951    3930       1
3891521   18951    3925       0
3891520   18951    3917       0
3891519   18951    3915       1
3891518   18951    3914       1
3891517   18951    3913       1
3891516   18951    3906       1
...     